In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
import ee
import geemap


ee.Initialize()

In [7]:
import sys
sys.path.append('../')

In [17]:
from sla.prep import PreProcessor
from sla.cloud_shadow_mask import add_cloud_shadow
from sla.classifier import decision_tree
from sla.hill_shadow import add_hill_shadow


In [18]:
ing_id = "G718255O411666S"
start_year = ee.Number(2022)
end_year = ee.Number(2022)
doy_start = ee.Number(50)
doy_end = ee.Number(200)
cloudiness = ee.Number(50)
coverage = 50
hsboolean = 0
hs_start = ee.Number(50)
hs_end = ee.Number(200)
dem = "SRTM"

preprocessor = PreProcessor(
    ing_id=ing_id,
    start_year=start_year,
    end_year=end_year,
    doy_start=doy_start,
    doy_end=doy_end,
    cloudiness=cloudiness,
    coverage=coverage,
    hsboolean=hsboolean,
    dem=dem,
)

preprocessed = preprocessor.execute()

print("processing cloud shadow mask...")
preprocessed = preprocessed.map(add_cloud_shadow)

print("processing hill shadow...")
preprocessed = add_hill_shadow(
        image_collection=preprocessed, hs_start=hs_start, hs_end=hs_end
    )

print("initiating classifier...")
map_collection = preprocessed.map(decision_tree)

processing cloud shadow mask...
processing hill shadow...
initiating classifier...


In [21]:
image = map_collection.first()

In [23]:
image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 8},
   'crs': 'EPSG:32719',
   'crs_transform': [10, 0, 199980, 0, -10, 5500000]}],
 'properties': {'deminfo': 'SRTM',
  'otsu': 0.47,
  'ING_ID': 'G718255O411666S',
  'system_time_start': 1647009866834,
  'system:index': '2_2_20220311T142741_20220311T144201_T19GBQ'}}

In [24]:
Map = geemap.Map(center=[40,-100], zoom=10)
Map.addLayer(image)
Map.centerObject(ee_obj, zoom=10)
Map

Map(center=[-41.16634439383657, -71.82854320247421], controls=(WidgetControl(options=['position', 'transparent…